In [15]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import optuna
import mlflow
from sklearn.model_selection import GridSearchCV
from pmdarima.arima import auto_arima
from keras.models import load_model
import joblib
from weatherapi import WeatherPoint
import requests
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pickle
from pmdarima.arima import auto_arima
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from keras.models import load_model
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import xgboost as xgb

#root_path = "/projects/app" #это если в контейнере 
root_path = "."  #это если в локальной машине
path_data = f'{root_path}/data'
your_mlflow_tracking_uri = f'{root_path}/mlruns'

mlflow.set_tracking_uri(your_mlflow_tracking_uri)


from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import optuna
import mlflow
from sklearn.model_selection import GridSearchCV
from pmdarima.arima import auto_arima
from keras.models import load_model
import joblib
from weatherapi import WeatherPoint
import requests
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pickle
from pmdarima.arima import auto_arima
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from keras.models import load_model
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import xgboost as xgb

In [8]:
data = pd.read_excel('data/archive.xls')


In [11]:
data.head()

,Local time in Moscow,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,13.02.2025 18:00,-7.4,750.9,765.4,-1.1,75.0,Wind blowing from the west,1.0,NaN,NaN,...,NaN,NaN,20,-11.2,Trace of precipitation,12.0,NaN,NaN,NaN,NaN
1,13.02.2025 15:00,-7.1,752.0,766.6,-1.2,80.0,Wind blowing from the west,1.0,NaN,NaN,...,NaN,NaN,18,-10.0,NaN,NaN,NaN,NaN,NaN,NaN
2,13.02.2025 12:00,-7.1,753.2,767.8,-0.9,83.0,Wind blowing from the west-northwest,1.0,NaN,NaN,...,NaN,NaN,17,-9.5,NaN,NaN,NaN,NaN,NaN,NaN
3,13.02.2025 09:00,-7.6,754.1,768.8,-0.8,86.0,Wind blowing from the west,1.0,NaN,NaN,...,NaN,NaN,14,-9.5,Trace of precipitation,12.0,NaN,NaN,Even layer of loose dry snow covering ground c...,1
4,13.02.2025 06:00,-7.8,754.9,769.6,-1.1,86.0,Wind blowing from the west,1.0,NaN,NaN,...,NaN,NaN,18,-9.8,No precipitation,12.0,NaN,NaN,NaN,NaN


In [14]:
data.DD.unique()

array(['Wind blowing from the west',
       'Wind blowing from the west-northwest',
       'Wind blowing from the west-southwest',
       'Wind blowing from the south-west', 'Calm, no wind',
       'Wind blowing from the north', 'Wind blowing from the north-east',
       'Wind blowing from the north-northwest',
       'Wind blowing from the east',
       'Wind blowing from the north-northeast',
       'Wind blowing from the east-northeast',
       'Wind blowing from the east-southeast',
       'Wind blowing from the south-southwest',
       'Wind blowing from the south',
       'Wind blowing from the south-southeast',
       'Wind blowing from the south-east',
       'Wind blowing from the north-west', 'variable wind direction', nan],
      dtype=object)

In [ ]:
import pandas as pd
import numpy as np

def convert_cloud_to_nh(cloud_percent):
    """Преобразует процент облачности (0-100) в категории Nh архива"""
    if cloud_percent == 100:
        return '100%.'
    elif 90 <= cloud_percent < 100:
        return '90  or more, but not 100%'
    elif 40 <= cloud_percent < 50:
        return '40%.'
    elif 20 <= cloud_percent < 30:
        return '20–30%.'
    elif 70 <= cloud_percent < 80:
        return '70 – 80%.'
    elif 50 <= cloud_percent < 60:
        return '50%.'
    elif cloud_percent == 0:
        return 'no clouds'
    elif 60 <= cloud_percent < 70:
        return '60%.'
    elif 0 < cloud_percent <= 10:
        return '10%  or less, but not 0'
    else:
        return np.nan

def convert_wind_dir(wind_dir):
    """Сопоставляет направления ветра из API с архивными описаниями"""
    direction_map = {
        'W': 'Wind blowing from the west',
        'WNW': 'Wind blowing from the west-northwest',
        'WSW': 'Wind blowing from the west-southwest',
        'SW': 'Wind blowing from the south-west',
        'N': 'Wind blowing from the north',
        'NE': 'Wind blowing from the north-east',
        'NNW': 'Wind blowing from the north-northwest',
        'E': 'Wind blowing from the east',
        'NNE': 'Wind blowing from the north-northeast',
        'ENE': 'Wind blowing from the east-northeast',
        'ESE': 'Wind blowing from the east-southeast',
        'SSW': 'Wind blowing from the south-southwest',
        'S': 'Wind blowing from the south',
        'SSE': 'Wind blowing from the south-southeast',
        'SE': 'Wind blowing from the south-east',
        'NW': 'Wind blowing from the north-west',
        'CALM': 'Calm, no wind',
        'VAR': 'variable wind direction'
    }
    return direction_map.get(wind_dir.upper(), np.nan)

def convert_api_to_archive(df_api):
    """Основная функция преобразования"""

    # Создаем пустой датафрейм с колонками как в архиве
    archive_columns = [
        'index', 'datetime', 'T', 'P0', 'P', 'U', 'DD', 'FF', 'FF10', 'FF3',
        'Tn', 'Tx', 'Nh', 'H', 'VV', 'RRR', 'sss', 'other_columns...'
    ]

    df_archive = pd.DataFrame(columns=archive_columns)

    # Основные преобразования
    df_archive['T'] = df_api['Temperature_C']
    df_archive['U'] = df_api['Humidity']
    df_archive['FF'] = df_api['Wind_Speed_kph'] * 0.27778  # км/ч -> м/с
    df_archive['P'] = df_api['Pressure_mb'] * 0.750062     # мбар -> мм рт.ст.
    df_archive['VV'] = df_api['Visibility_km']

    # Преобразование направления ветра
    df_archive['DD'] = df_api['Wind_Direction'].apply(convert_wind_dir)

    # Преобразование облачности
    df_archive['Nh'] = df_api['Cloud'].apply(convert_cloud_to_nh)

    # Дата и время
    df_archive['datetime'] = pd.to_datetime(df_api['Localtime'])

    # Заполнение недостающих колонок
    df_archive['P0'] = np.nan  # Нет данных в API
    df_archive['FF10'] = df_api.get('wind_gust_kph', np.nan) * 0.27778
    df_archive['FF3'] = np.nan
    df_archive['Tn'] = np.nan
    df_archive['Tx'] = np.nan
    df_archive['H'] = np.nan
    df_archive['RRR'] = df_api.get('precipitation_mm', np.nan)
    df_archive['sss'] = df_api.get('snow_cm', np.nan)

    # Форматирование под архив
    df_archive = df_archive.replace({np.nan: 'NaN'})

    # Приведение к правильному порядку колонок
    df_archive = df_archive[archive_columns]

    return df_archive



### Запрос API

In [ ]:
# Установка базового URL и ключа API
base_url = "http://api.weatherapi.com/v1"
api_key = "ae03964b347b4881afe205155242404"  # Замените YOUR_API_KEY на ваш ключ API

# Параметры запроса
params = {
    "key": api_key,
    "q": "Moscow"  # Название города, для которого мы хотим получить погоду
}

# Формирование запроса
response = requests.get(f"{base_url}/current.json", params=params)